<a id="bqlc-top"></a>
<div style="width: 50px; height: 50px; border-radius: 50%; background: #F1C40F; margin-bottom: 0px; text-align: center;">
    <img src="../img/Welcome Page/launch.png" style="max-height: 95%; max-width: 95%;">
</div>
<h1 style="margin-top: 20px; margin-bottom: 5px;">BQL Concepts<br><span style="color: orange; margin-bottom: 0px;">Analyzing Data with BQL</span></h1>
<h4 style="color: orange; margin-top: 0px;">(20-min Read)</h4>

By the end of this document, you will be able to:
1. Chain BQL items to form complex queries
1. Effectively use the `members()` and `filter()` universe handlers
1. Understand the `group()` function for aggregating data
1. Perform a few statistical calculations in BQL

### Contents
- [The BQuant Project Flow](#project-flow-bqlc)
- [BQL Refresh](#bqlc-ref)
- [Chaining BQL Objects](#bqlc-chain)
- [Universe Handlers](#bqlc-univ)
- [Aggregate Data with the `group()` function](#bqlc-data)
- [Using the BQL Editor](#bqlc-editor)
- [The BQL Reference Guide](#bqlc-guide)

<a href="../1. Boot Camp/4 Boot Camp Project.ipynb">&larr; Back to Boot Camp: Final Project</a>&emsp; | &emsp;
<a href="2 DataGrids.ipynb">Continue to Launch: DataGrids &rarr;</a><br>
<a href="../Welcome.ipynb#welcome-top" style="font-size: 12px;">Return to the Welcome Page</a>

---
<a id="project-flow-bqlc"></a>
<h1>BQuant Project Flow<br><span style="color: orange;">5-Step Process for Developing Your Projects</span></h1>

The 5-step project flow for BQuant development was mentioned in the Intro to BQuant section, but it bears repeating here. Every project you develop in BQuant will follow the same basic process below.

1. Setup Environment
1. Get Data from Bloomberg using BQL
1. Additional Data Munging (if necessary)
1. User Interface & Visualizations
1. Simplify, Document, & Publish

In this notebook, we cover functions and universes of the Bloomberg Query Language (BQL) for use in Steps 1 and 2.

[&uarr; Return to Top](#bqlc-top)

---
<a id="bqlc-ref"></a>
<h1>BQL Refresh<br><span style="color: orange;">Getting Data From Bloomberg</span></h1>

The Bloomberg Query Language (BQL) is the most effective way to retrieve, manipulate, and analyze data in the BQuant environment. You can think of BQL as your access point to Bloomberg's wealth of information. Inside the BQuant environment, you will use BQL to send requests for information to Bloomberg servers and receive a packet of data in return. Bloomberg enginners have created the `bql` Python library for building queries and interacting with the BQL API. At the beginning of every project, `import bql` and establish a connection to Bloomberg servers by initializing a Service object with `bq = bql.Service()`.

All BQL requests need at least two arguments: the type of data, and the universe. You can think of BQL queries as having two statements: `GET` and `FOR`. If you need daily trading volume for Apple stock, you are asking BQL to "get" volume data "for" Apple. All BQL queries will follow this basic structure: `GET <data type> FOR <universe>`. In the cell below, we retrieve closing prices of the CAC 40 Index for the last 5 days.


In [1]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
import bql
bq = bql.Service()

# Step 2: Get Data with BQL
universe = 'FR Index'
date_range = bq.func.range('-5D', '0D')
prices = bq.data.px_last(dates=date_range)
req = bql.Request(universe, {'Closing Prices': prices})   # <-- build request; GET prices FOR universe
res = bq.execute(req)

# display result as a DataFrame
res[0].df()

,DATE,CURRENCY,Closing Prices
ID,,,
FR Index,2021-04-03,USD,NaN
FR Index,2021-04-04,USD,NaN
FR Index,2021-04-05,USD,1332.15
FR Index,2021-04-06,USD,1342.08
FR Index,2021-04-07,USD,1349.23
FR Index,2021-04-08,USD,1352.16


[&uarr; Return to Top](#bqlc-top)

---
<a id="bqlc-chain"></a>
<h1>Chaining BQL Objects<br><span style="color: orange;">Performing Functions on Queries</span></h1>

When you initialize a BQL item, either by calling `bq.func` or `bq.data`, behind the scenes Python creates an instance of a special class called `BqlItemFactory`. You can see this yourself by clicking on `range` or `px_last` above and using the key combination `Shift + Tab` to show BQuant's Contextual Help.

<h5 style="color: orange; margin-bottom: 0px;">Image: BQuant Contextual Help</h5>
<img src="../img/Launch/contextual_help.png">

You don't need to know all the details of how the `BqlItemFactory` class works, but it's important to keep in mind that all of these objects are of the same type. When you run `bq.func.range()`, you create the same type of object as when you run `bq.data.px_last()`. All BQL items can build on each other, meaning you don't need to explicitly call `bq.data` or `bq.func` to perform a calculation on an already existing BQL item. We can "chain" these items together to build complex queries.

Consider the code examples below. These will return identical results, but the second example is much cleaner.

<h4 style="color: orange;">Code Example 1</h4>

In [2]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
# import bql
# bq = bql.Service()

# Step 2: Get Data with BQL
universe = 'FR Index'
prices = bq.data.px_last('-5D', '0D')
drop_null_values = bq.func.dropna
drop_missing_prices = drop_null_values(prices)

# build & execute Request
req = bql.Request(universe, {'Closing Prices': drop_missing_prices})
res = bq.execute(req)

# display as DataFrame
res[0].df()


,DATE,CURRENCY,Closing Prices
ID,,,
FR Index,2021-04-05,USD,1332.15
FR Index,2021-04-06,USD,1342.08
FR Index,2021-04-07,USD,1349.23
FR Index,2021-04-08,USD,1352.38


<h4 style="color: orange;">Code Example 2 - Chain the BQL items</h4>

In [3]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
# import bql
# bq = bql.Service()

# Step 2: Get Data with BQL
universe = 'FR Index'
prices = bq.data.px_last('-5D', '0D').dropna()    # <-- chain BQL items together

# build & execute Request
req = bql.Request(universe, {'Closing Prices': drop_missing_prices})
res = bq.execute(req)

# display as DataFrame
res[0].df()


,DATE,CURRENCY,Closing Prices
ID,,,
FR Index,2021-03-12,USD,1336.70
FR Index,2021-03-15,USD,1331.53
FR Index,2021-03-16,USD,1333.29
FR Index,2021-03-17,USD,1333.57


In the above example, we chained BQL items together to perform a function on a data item. Chaining here saved us a few lines of code; that might not seem like much for a simple query like this, but get in the habit of chaining BQL items together to save yourself a lot of work for more complex queries!

<div style="border: 1px dotted orange; padding-left: 10px; padding-bottom: 10px;">
    <h3>&#9998; Try it out</h3>
    <p>
        Practice Chaining BQL Items
        <ol>
            <li>Add a new Jupyter code cell below this one</li>
            <li>Declare a <code>universe</code> variable and set it equal to the Bloomberg ticker for your favorite equity index.</li>
            <li>Declare a second variable <code>prices</code> to retrieve the last 30 days of closing prices.</li>
            <li>Declare a third variable named <code>avg_price</code> and chain the <code>bq.func.avg()</code> item onto the end of the <code>prices</code> variable you declared in Step 3. This will calculate the average price over the time series.</li>
            <li>Build and execute a request for <code>avg_price</code>. Display the results as a DataFrame. If successful, the query should only return one row.</li>
        </ol>
    We've written one possible solution to this problem in a hidden Python cell below. When you're ready to get a peek at an answer, click the <img src="../img/Controls/hidden_cell.png"> below.
    </p>
</div>

<p style="color: orange; font-size: 12px;">&darr; Click below to expand answer</p>

In [ ]:
# import bql
# bq = bql.Service()

# Step 2
universe = 'JP Index'  # <-- Bloomberg Japan Large & Mid Cap Price Return Index USD 

# Step 3
prices = bq.data.px_last('-30D', '0D')

# Step 4
avg_price = prices.avg()

# Step 5
request = bql.Request(universe, {'Average 30D Price': avg_price})
response = bq.execute(request)
df = response[0].df()
df

[&uarr; Return to Top](#bqlc-top)

---
<a id="bqlc-univ"></a>
<h1>Universe Handlers<br><span style="color: orange;">Refine Queries with <code style="color: orange; background-color: transparent;">members()</code> and <code style="color: orange; background-color: transparent;">filter()</code></span></h1>

In most of your BQL queries, the `GET` argument will be more complex than the `FOR` universe. However, there are a few special functions built for universes called "universe handlers." The most common universe handlers are `members()` and `filter()`. 

  - `members()` allows you to access constituent data on an index, portfolio, or fund. BQL can then pull data or run functions on the underlying members.
  - `filter()` will limit the results returned based on given criteria

<h4 style="color: orange;">Members of an Index</h4>
<p>The code below uses the <code>members()</code> handler to pull the most recent price for every member of the Bloomberg US Large Cap Index.</p>

In [7]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
# import bql
# bq = bql.Service()

# Step 2: Get Data with BQL
b500_members = bq.univ.members('B500T Index')
latest_price = bq.data.px_last(fill='prev')       # <-- use fill='prev' to use the most recently available price if the stock has not traded recently
req = bql.Request(b500_members, {'Price': latest_price})
res = bq.execute(req)

# display first 5 rows of DataFrame
res[0].df().head()

,DATE,CURRENCY,Price
ID,,,
LYB UN Equity,2021-03-17,USD,108.250000
AXP UN Equity,2021-03-17,USD,144.229996
VZ UN Equity,2021-03-17,USD,55.790001
AVGO UW Equity,2021-03-17,USD,485.720001
BA UN Equity,2021-03-17,USD,261.470001


<h4 style="color: orange;">Members of a Fund</h4>
<p>Use <code>type='holdings'</code> and <code>bq.func.id()</code> to access holdings data of an ETF.</p>

In [14]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
# import bql
# bq = bql.Service()

# Step 2: Get Data with BQL
fund = bq.univ.members('SPY US Equity', type='holdings')
req = bql.Request(fund, bq.data.id())
res = bq.execute(req)

# display first 5 rows of DataFrame
res[0].df().head()


,POSITION,WEIGHTS,AS_OF_DATE,REPORTED_MKT_VAL,CURRENCY,LATEST_CHANGE,ORIG_IDS,ID()
ID,,,,,,,,
AAPL US Equity,171835854.0,7.438908,2020-11-24,1.979034e+10,USD,811020.0,SPY US Equity,AAPL US Equity
MSFT US Equity,80888141.0,6.502352,2020-11-24,1.729874e+10,USD,381780.0,SPY US Equity,MSFT US Equity
AMZN US Equity,4550976.0,5.333902,2020-11-24,1.419022e+10,USD,21504.0,SPY US Equity,AMZN US Equity
FB US Equity,25698610.0,2.674977,2020-11-24,7.116459e+09,USD,121296.0,SPY US Equity,FB US Equity
GOOGL US Equity,3211411.0,2.129246,2020-11-24,5.664608e+09,USD,15120.0,SPY US Equity,GOOGL US Equity


<h4 style="color: orange;">Filter</h4>
<p>Use the <code>filter()</code> function in combination with <code>members()</code> to retrieve P/E ratios for stocks in the Consumer Staples sector.

In [8]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
# import bql
# bq = bql.Service()

# Step 2: Get Data with BQL
sector = bq.data.gics_sector_name()
b500_members = bq.univ.members('B500T Index')
pe = bq.data.pe_ratio(fill='prev')
staples_only = b500_members.filter(sector == 'Consumer Staples')

req = bql.Request(staples_only, {'P/E Ratio': pe})
res = bq.execute(req)

# display first 5 rows of DataFrame
res[0].df().head()


,AS_OF_DATE,REVISION_DATE,PERIOD_END_DATE,P/E Ratio
ID,,,,
KO UN Equity,2021-03-17,2021-02-25,2020-12-31,26.329439
PG UN Equity,2021-03-17,2021-01-20,2020-12-31,22.871585
WMT UN Equity,2021-03-17,2021-02-18,2021-01-31,23.965787
MO UN Equity,2021-03-17,2021-02-17,2020-12-31,11.434510
ADM UN Equity,2021-03-17,2021-02-18,2020-12-31,15.498700


<div style="border: 1px dotted orange; padding-left: 10px; padding-bottom: 10px;">
    <h3>&#9998; Try it out</h3>
    <p>
        Practice with <code>members()</code> and <code>filter()</code>
        <ol>
            <li>Add a new Jupyter code cell below this one</li>
            <li>Declare a variable named <code>universe</code> and set it equal to the members of the <code>'BWORLD Index'</code>.</li>
            <li>Write a query to pull the most recent price for members of that index domiciled in Brazil (BR). Hint: use the BQL Editor to find the correct field</li>
        </ol>
    We've written one possible solution to this problem in a hidden Python cell below. When you're ready to get a peek at an answer, click the <img src="../img/Controls/hidden_cell.png"> below.
    </p>
</div>

<p style="color: orange; font-size: 12px;">&darr; Click below to expand answer</p>

In [ ]:
# import bql
# bq = bql.Service()

# Step 2
universe = bq.univ.members('BWORLD Index')    # <-- get the members of the index rather than the index itself

# Step 3
last_price = bq.data.px_last(fill='PREV')   # <-- fill='PREV' will return the most recent market price, ignoring non-trading days
country = bq.data.cntry_of_domicile()
brazil_stocks = bq.univ.filter(universe, country == 'BR')

request = bql.Request(brazil_stocks, {'Closing Price': last_price})
response = bq.execute(request)
df = response[0].df()
df.head()

[&uarr; Return to Top](#bqlc-top)

---
<a id="bqlc-data"></a>
<h1>Aggregate Data<br><span style="color: orange;">Using the <code style="color: orange; background-color: transparent;">group()</code>function</span></h1>

Consider the code example below. We're trying to find the average P/E ratio for all members of the Bloomberg Europe 500 Index.

In [10]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
# import bql
# bq = bql.Service()

# Step 2: Get Data with BQL
be500_members = bq.univ.members('BE500 Index')
pe = bq.data.pe_ratio(fill='prev')
avg_pe = pe.avg()

req = bql.Request(be500_members, {'Average P/E': avg_pe})
res = bq.execute(req)

# display 5 rows of the result
res[0].df().head()

,REVISION_DATE,PERIOD_END_DATE,AS_OF_DATE,Average P/E
ID,,,,
ENX FP Equity,2021-02-10,2020-12-31,2021-03-17,17.237364
BOL SS Equity,2021-03-10,2020-12-31,2021-03-17,12.910320
EVD GR Equity,2020-11-19,2020-09-30,2021-03-17,NaN
BNR GR Equity,2021-03-10,2020-12-31,2021-03-17,21.680090
LOGN SW Equity,2021-01-21,2020-12-31,2021-03-17,14.794507


Not quite what you were expecting? The BQL request should have only returned one row since the average P/E of all members is just one number. Instead, the query as it's written above applies the `pe.avg()` function to every stock in the index. If "Stock A" has a P/E of 10, BQL is just taking the average of one number (10). It repeats this behavior for every member of the index. In fact, this query is exactly the same as just requesting `pe` without `avg()`.

How do we fix this? Meet the `group()` function. It allows you aggregate values and perform functions on a set. Grouping the members before calculating the average will give us the results we need. Let's rewrite the query below.

In [11]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
# import bql
# bq = bql.Service()

# Step 2: Get Data with BQL
be500_members = bq.univ.members('BE500 Index')
pe = bq.data.pe_ratio(fill='prev')
avg_pe = pe.group().avg()           # <-- group the constituents and calculate the average of the group

req = bql.Request(be500_members, {'Average P/E': avg_pe})
res = bq.execute(req)

# display the result
res[0].df()

,REVISION_DATE,PERIOD_END_DATE,AS_OF_DATE,ORIG_IDS,Average P/E
ID,,,,,
IdentityGroup,2021-03-17,2021-01-31,2021-03-17,None,50.992735


<h4 style="color: orange;">Group By Specific Criteria</h4>
<p>Passing no arguments to <code>group()</code> will group everything in one set. However, we can refine our groupings by passing in some conditions. The code below calculates the average P/E ratio by sector for members of the S&P 500 Index. </p>

In [12]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
# import bql
# bq = bql.Service()

# Step 2: Get Data with BQL
be500_members = bq.univ.members('BE500 Index')
sector = bq.data.gics_sector_name()
pe = bq.data.pe_ratio(fill='prev')
avg_pe_by_sector = pe.group(sector).avg()      # <-- pass in sector as an argument for group()

req = bql.Request(be500_members, {'Average P/E': avg_pe_by_sector})
res = bq.execute(req)

# display the result
res[0].df()

,REVISION_DATE,PERIOD_END_DATE,AS_OF_DATE,ORIG_IDS,GICS_SECTOR_NAME(),Average P/E
ID,,,,,,
Communication Services,2021-03-11,2020-12-31,2021-03-17,None,Communication Services,91.002174
Consumer Discretionary,2021-03-16,2021-01-31,2021-03-17,None,Consumer Discretionary,70.856635
Consumer Staples,2021-03-11,2021-01-31,2021-03-17,None,Consumer Staples,26.470104
Energy,2021-03-11,2020-12-31,2021-03-17,None,Energy,71.610512
Financials,2021-03-15,2020-12-31,2021-03-17,None,Financials,17.311391
Health Care,2021-03-16,2021-01-31,2021-03-17,None,Health Care,47.217591
Industrials,2021-03-16,2021-01-31,2021-03-17,None,Industrials,85.905229
Information Technology,2021-03-09,2020-12-31,2021-03-17,None,Information Technology,50.778027
Materials,2021-03-16,2020-12-31,2021-03-17,None,Materials,32.147989


<a id="dg-link-here"></a>
<div style="border: 1px dotted orange; padding-left: 10px; padding-bottom: 10px;">
    <h3>&#9998; Try it out</h3>
    <p>
        Practice with <code>group()</code>
        <ol>
            <li>Add a new Jupyter code cell below this one</li>
            <li>Declare a variable named <code>universe</code> and set it equal to the members of the <code>'BWORLD Index'</code>.</li>
            <li>Write a query to calcuate the average P/E by <code>country_full_name</code>.</li>
            <li>For an extra challenge, use BQL's <code>sort()</code> function to sort the dataframe in descending order. Which country's stocks are the most expensive?<br><span style="color: orange;">Hint:</span> use the <code>group()</code> function again on the results to sort the entire set.</li>
        </ol>
    We've written one possible solution to this problem in a hidden Python cell below. When you're ready to get a peek at an answer, click the <img src="../img/Controls/hidden_cell.png"> below.
    </p>
</div>

<p style="color: orange; font-size: 12px;">&darr; Click below to expand answer</p>

In [ ]:
# import bql
# bq = bql.Service()

# setup the BQL query
universe = bq.univ.members('BWORLD Index')
pe_ratio = bq.data.pe_ratio(fill='prev')
country = bq.data.country_full_name()
avg_pe_by_country = pe_ratio.group(country).avg()
sorted_pe_by_country = avg_pe_by_country.group().sort(order='desc')   # <-- group again and sort the group

# Make Request & Parse Response
request = bql.Request(universe, {'Avg PE by Country': sorted_pe_by_country})
response = bq.execute(request)
df = response[0].df()

# Clean the dataframe
df.set_index('COUNTRY_FULL_NAME()', inplace=True)  # <-- set the index to the country name
df = df[['Avg PE by Country']]    # <-- drop unnecessary columns 
df.head()     # <-- display the first five rows

[&uarr; Return to Top](#bqlc-top)

---
<a id="bqlc-editor"></a>
<h1>Using the BQL Editor Tool<span style="color: orange;"><br>Search Fields, Universes, and Functions</span></h1>

BQuant provides a useful tool called the "BQL Editor" to help you write and implement BQL queries. 

The BQL Editor allows you to:
   1. Search for Fields, Functions, and Universes
   1. Run a test query and get a preview of the results
   1. Copy the query's Python code so you can paste it in a Jupyter cell
   
You can access the BQL Editor in the toolbar at the top of the screen. Clicking the BQL Editor button will open up a separate tab in which you can write your test query.

The example below demonstrates the use of the BQL Editor in building a query for the most recent S&P 500 price.

<img src="../img/Boot Camp/bql_editor.gif">

[&uarr; Return to Top](#bqlc-top)

---
<a id="bqlc-guide"></a>
<h1>The BQL Reference Guide<br><span style="color: orange;">Leverage Existing Documentation</span></h1>

BQL offers a suite of mathematical, statistical, and logistic functions allowing you to perform complex calculations on the server side rather than using the resources on your local computer. BQuant's Example Projects include a reference guide for commonly used BQL funcitons and universes. Find it in <a href="../exampleroot/1a54e18169d24618afc2226346108fb4/0 Welcome.ipynb">Example Projects &rarr; BQL Resources &rarr; BQL Function and Universe Guide</a>. Use this guide and the BQL Editor to write the perfect BQL query for your project.

[&uarr; Return to Top](#bqlc-top)

----
<p style="text-align:center;">
    Click on the links below to continue learning.<br>
    <a href="../1. Boot Camp/4 Boot Camp Project.ipynb">&larr; Back to Boot Camp: Final Project</a>&emsp;&emsp;
    <a href="#bqlc-top">&uarr; Return to Top</a>&emsp;&emsp;
    <a href="2 DataGrids.ipynb">Continue to Launch: DataGrids &rarr;</a>
    <br>
    <br>
    <a href="../Welcome.ipynb#welcome-top" style="font-size: 12px;">Return to the Welcome Page</a>
</p>